In [5]:
import pandas as pd

# 複数シートのデータを1つのデータフレームに格納する
iter_num = [i for i in range(0, 3)]

df = pd.DataFrame()

for i in iter_num:
    each_df = pd.read_excel("sample_4.xlsx", index_col=0, sheet_name=i)
    df = pd.concat([df, each_df], axis=0)  # axis=0:行

df.head()
df.tail()

,曜日,目的地,運行回数,座席数,貨物重量,旅客数
日付,,,,,,
2016-12-27,火,OKA,63,20249,441533,14401
2016-12-28,水,OKA,63,20203,441506,14334
2016-12-29,木,OKA,60,20336,441919,14035
2016-12-30,金,OKA,63,20077,441831,14714
2016-12-31,土,OKA,62,20879,441586,14012


In [9]:
# 新規の列「運行回数当たりの旅客数」を算出
df.loc[:, '旅客数/運行回数'] = (df.loc[:, '旅客数'] / df.loc[:, '運行回数']).round()
df.head()

,曜日,目的地,運行回数,座席数,貨物重量,旅客数,旅客数/運行回数
日付,,,,,,,
2016-12-01,木,CTS,93,29638,516139,20260,218.0
2016-12-02,金,CTS,100,29872,516287,20211,202.0
2016-12-03,土,CTS,92,29568,516120,20541,223.0
2016-12-04,日,CTS,92,29456,516960,20747,226.0
2016-12-05,月,CTS,94,29896,516062,20091,214.0


In [12]:
from sklearn.cluster import KMeans

# クラスタ数を4つにしてインスタンスを作成する
KM = KMeans(
    n_clusters=4,  # クラスタ数 デフォルト=8
    init="random",  # 初期化方法
    n_init=100,  # k-meansの実行回数
    random_state=0,  # 乱数のシード値を固定するための引数
)

x = df.iloc[:, 2:]

cluster = KM.fit_predict(x)

df.loc[:, "cluster"] = cluster  # 予測結果をデータフレームに追加
df.sort_values('日付').head()

,曜日,目的地,運行回数,座席数,貨物重量,旅客数,旅客数/運行回数,cluster
日付,,,,,,,,
2016-12-01,木,CTS,93,29638,516139,20260,218.0,2
2016-12-01,木,OKA,63,20743,441477,14047,223.0,3
2016-12-01,木,ITM,59,19814,223345,15262,259.0,1
2016-12-02,金,CTS,100,29872,516287,20211,202.0,2
2016-12-02,金,OKA,70,20350,441090,14117,202.0,3


In [16]:
df.query(expr='cluster == 0').head(3)
df.to_excel('cluster.xlsx')

# 回帰を使って予測をする
1. 「何を予測したいか」目的変数を決める
2. 目的変数との相関が高い説明変数を選定
3. 散布図で可視化（スキップ可）
4. train_test_splitクラスを使い実データを学習データとテストデータに分割する
5. 予測クラスを決める
6. fitメソッドで学習し、predictメソッドで予測をする
7. 予測結果とテストデータを突合する（答え合わせ）
8. 実際に未知のデータで予測する

In [18]:
cts = df[df.loc[:, '目的地'] == 'CTS']
itm = df[df.loc[:, '目的地'] == 'ITM']
oka = df[df.loc[:, '目的地'] == 'OKA']

cts.describe()

,運行回数,座席数,貨物重量,旅客数,旅客数/運行回数,cluster
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,95.000000,29497.225806,516499.580645,20509.096774,216.000000,1.032258
std,2.977695,295.150437,323.254159,272.681543,7.456541,1.016001
min,91.000000,29034.000000,516033.000000,20032.000000,202.000000,0.000000
25%,92.000000,29243.500000,516195.500000,20305.000000,210.000000,0.000000
50%,95.000000,29470.000000,516488.000000,20541.000000,217.000000,2.000000
75%,98.000000,29755.000000,516794.000000,20645.000000,221.000000,2.000000
max,100.000000,29970.000000,516983.000000,20968.000000,230.000000,2.000000
